In [1]:
%sh pip install bs4

Requirement already satisfied: bs4 in /databricks/python3/lib/python3.7/site-packages (0.0.1)
Requirement already satisfied: beautifulsoup4 in /databricks/python3/lib/python3.7/site-packages (from bs4) (4.9.1)
Requirement already satisfied: soupsieve>1.2 in /databricks/python3/lib/python3.7/site-packages (from beautifulsoup4->bs4) (2.0.1)
WARNING: You are using pip version 20.0.2; however, version 20.2.3 is available.
You should consider upgrading via the '/databricks/python3/bin/python3.7 -m pip install --upgrade pip' command.

In [2]:
from bs4 import BeautifulSoup
import requests

In [3]:
import pandas as pd
import datetime

In [4]:
url="https://healthpayerintelligence.com/tag/medicare-advantage"

In [5]:
#Extracting headline and article links 
news_dict=[]
feature=[]
response=requests.get(url)
soup=BeautifulSoup(response.content)

for link1 in soup.find_all("section",{"id":"featured-article"}):
  feature.append({'headline': link1.text,'article link': link1.find_all('a')[0].get('href'),'post_date': link1.find_all('span')[0].get('slider-date')})

for link in soup.find_all("div",{"class":"col-lg-12 cl-md-12 col-sm-12 col-xs-12"}):
  #print(link)
  if(len(link.text.split(" "))>4):
    news_dict.append({'headline': link.text,'article link': link.find_all('a')[0].get('href'),'post_date': link.find_all('time')[0].get('datetime')})

In [6]:
newsfeed_df=pd.DataFrame(news_dict)
feature_df=pd.DataFrame(feature)

In [7]:
#Appending the featured article to the newsfeed dataframe
newsfeed_df=feature_df.append(newsfeed_df)

In [8]:
newsfeed_df=newsfeed_df.reset_index()
newsfeed_df=newsfeed_df.drop(['index'],axis=1)

In [9]:
#Extracting the date of the featured article
feature_url=newsfeed_df['article link'][0]

response1=requests.get(feature_url)
soup1=BeautifulSoup(response1.content)

for fl in soup1.find_all("main",{"id":"articles-content"}):
  f_date=fl.find_all('time')[0].get('datetime')
  
newsfeed_df['post_date'][0]=f_date

In [10]:
#Extracting only the needed headline part

newsfeed_df['headline']=newsfeed_df['headline'].str.split('\n').str[1]

In [11]:
newsfeed_df['post_date']=pd.to_datetime(newsfeed_df['post_date'])

In [12]:
newsfeed_df.head()

,headline,article link,post_date
0,MA Special Needs Plans May Lower ESRD Spending...,https://healthpayerintelligence.com/news/ma-sp...,2020-09-10
1,How COVID-19 Policies May Affect Medicare Adva...,https://healthpayerintelligence.com/news/how-c...,2020-09-04
2,"Neither Anthem, Cigna to Receive Damages in Me...",https://healthpayerintelligence.com/news/neith...,2020-09-01
3,"How Payers Use Whole Person, Palliative Care t...",https://healthpayerintelligence.com/news/how-p...,2020-08-14
4,"Sentara Healthcare, Cone Health Announce Value...",https://healthpayerintelligence.com/news/senta...,2020-08-13


In [13]:
import datetime as DT
today = DT.date.today()
month_ago = today - DT.timedelta(days=30)
month_ago

Out[36]: datetime.date(2020, 8, 11)

In [14]:
#Checking of the post date is before the needed date(i.e week ago or 15 days)

newsfeed_df['past month']='a'
j=0
for i in newsfeed_df['post_date']:
  #temp=datetime.date(i)
  if(i>week_ago):
    newsfeed_df['past month'][j]='yes'
    j=j+1
  else:
    newsfeed_df['past month'][j]='no'
    j=j+1

/local_disk0/tmp/1599769781965-0/PythonShell.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 import resource
/local_disk0/tmp/1599769781965-0/PythonShell.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 import six.moves.queue

In [15]:
#Extracting only the articles posted in the past 15 days

req_df = newsfeed_df[newsfeed_df['past month'] == 'yes']

In [16]:
#Extracting the articles

article_dict=[]
for link in req_df['article link']:
  #print(link)
  response=requests.get(link)
  soup=BeautifulSoup(response.content)
  #print(response)

  for link1 in soup.find_all("main",{"id":"articles-content"}):
    #print("aa")
    article_dict.append({'article link': link,'article': link1.text})

In [17]:
article_df=pd.DataFrame(article_dict)

In [18]:
article_df.head()

,article link,article
0,https://healthpayerintelligence.com/news/ma-sp...,\n\n \n\n\n By Kelsey Waddill\n\n\nSeptember ...
1,https://healthpayerintelligence.com/news/how-c...,\n\n \n\n\n By Kelsey Waddill\n\n\nSeptember ...
2,https://healthpayerintelligence.com/news/neith...,\n\n \n\n\n By Kelsey Waddill\n\n\nSeptember ...


In [19]:
#Merging article and headline dataframes

merge_df=req_df.merge(article_df,how="outer")

In [20]:
#Dropping the not needed columns

merge_df=merge_df.drop(['past month'], axis=1)

In [21]:
merge_df

,headline,article link,post_date,article
0,MA Special Needs Plans May Lower ESRD Spending...,https://healthpayerintelligence.com/news/ma-sp...,2020-09-10,\n\n \n\n\n By Kelsey Waddill\n\n\nSeptember ...
1,How COVID-19 Policies May Affect Medicare Adva...,https://healthpayerintelligence.com/news/how-c...,2020-09-04,\n\n \n\n\n By Kelsey Waddill\n\n\nSeptember ...
2,"Neither Anthem, Cigna to Receive Damages in Me...",https://healthpayerintelligence.com/news/neith...,2020-09-01,\n\n \n\n\n By Kelsey Waddill\n\n\nSeptember ...


In [22]:
#The merge_df dataframe has all the articles posted in the past one month which are related to 'medicare advantage'